<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [3]:
import pandas as pd
import tweepy

In [4]:
##leer llaves

keys = pd.read_csv('app_tt.csv', header=None)
#se cnvierte en dic para realizar emparejamiento

keys = dict(zip(keys[0], keys[1])) #nuncaaa dejar las llaves visibles
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [5]:
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [6]:
tweets = tweepy.Cursor(api.search,
                       q=['champions league -RT'],
                       lang='es',
                       geocode='4.7110,-74.0721,500km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [7]:
t = [tweet for tweet in tweets]

In [8]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)

In [9]:
t_processed

[['Hoy, Fémina Fútbol España presente en la celebración del Barcelona femenino💙❤️ \nCampeonas de la Liga Iberdrola y la Champions League.\n@FCBfemeni https://t.co/ynLsCabBHG',
  datetime.datetime(2021, 5, 20, 1, 12, 21),
  'Bogotá',
  '',
  1395185595815714816,
  'https://twitter.com/i/web/stuatus/1395185595815714816'],
 ['⏱️ El LIVERPOOL gana por goleada y asegura su puesto en zona de CHAMPIONS LEAGUE frente al BURNLEY #PremierLeague https://t.co/84kZieDbs8',
  datetime.datetime(2021, 5, 20, 0, 38, 28),
  'Bogotá',
  'Cali, Colombia',
  1395177067919065089,
  'https://twitter.com/i/web/stuatus/1395177067919065089'],
 ['¡Pep Guardiola sigue agrandando su historia como entrenador! 🧠⚽️\n\nEl técnico español ganó la Premier League con el Manchester City, está en la final de la Champions League y continúa cosechando triunfos en su exitosa carrera como timonel. 🔝⚽️🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f🏆 https://t.co/BWCvfbR9z7',
  datetime.datetime(2021, 5, 20, 0, 34,

In [10]:
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()


,tweets,date,city,user_location,id,link
0,"Hoy, Fémina Fútbol España presente en la celeb...",2021-05-20 01:12:21,Bogotá,,1395185595815714816,https://twitter.com/i/web/stuatus/139518559581...
1,⏱️ El LIVERPOOL gana por goleada y asegura su ...,2021-05-20 00:38:28,Bogotá,"Cali, Colombia",1395177067919065089,https://twitter.com/i/web/stuatus/139517706791...
2,¡Pep Guardiola sigue agrandando su historia co...,2021-05-20 00:34:19,Bogotá,"Bogotá, Colombia",1395176020496588804,https://twitter.com/i/web/stuatus/139517602049...
3,Liverpool ganó y depende de sí para clasificar...,2021-05-20 00:19:04,Bogotá,"Bogotá, Colombia",1395172184029597699,https://twitter.com/i/web/stuatus/139517218402...
4,La Libertadores men. La puede ver en medio de ...,2021-05-20 00:17:01,Bogotá,"Ibagué, Colombia",1395171666590945284,https://twitter.com/i/web/stuatus/139517166659...


In [11]:
data.to_csv("taller11.csv", index=False)

In [12]:
##pre procesamiento
data = data[['tweets', 'date']]
data.head()

,tweets,date
0,"Hoy, Fémina Fútbol España presente en la celeb...",2021-05-20 01:12:21
1,⏱️ El LIVERPOOL gana por goleada y asegura su ...,2021-05-20 00:38:28
2,¡Pep Guardiola sigue agrandando su historia co...,2021-05-20 00:34:19
3,Liverpool ganó y depende de sí para clasificar...,2021-05-20 00:19:04
4,La Libertadores men. La puede ver en medio de ...,2021-05-20 00:17:01


In [13]:
import re
pattern1 = '?P<pic>pic.twitter.com/[^\s]+' #patron de regex
pattern2 = '?P<url>https?://[^\s]+' #patron de URL

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,texto_limpio,links,hashtags,mentions
0,"Hoy, Fémina Fútbol España presente en la celeb...",2021-05-20 01:12:21,"Hoy, Fémina Fútbol España presente en la celeb...",[https://t.co/ynLsCabBHG],[],[@FCBfemeni]
1,⏱️ El LIVERPOOL gana por goleada y asegura su ...,2021-05-20 00:38:28,⏱️ El LIVERPOOL gana por goleada y asegura su ...,[https://t.co/84kZieDbs8],[#PremierLeague],[]
2,¡Pep Guardiola sigue agrandando su historia co...,2021-05-20 00:34:19,¡Pep Guardiola sigue agrandando su historia co...,[https://t.co/BWCvfbR9z7],[],[]
3,Liverpool ganó y depende de sí para clasificar...,2021-05-20 00:19:04,Liverpool ganó y depende de sí para clasificar...,[https://t.co/y3Lx3wQEhu],[],[]
4,La Libertadores men. La puede ver en medio de ...,2021-05-20 00:17:01,La Libertadores men. La puede ver en medio de ...,[],[],[]
...,...,...,...,...,...,...
95,@alm4perdida @maores13 @ElPetisoArango @Atleti...,2021-05-17 16:39:53,"Ojo, Villarreal aún no ha clasificado a E...",[],[],"[@alm4perdida, @maores13, @ElPetisoArango, @At..."
96,Estreno de lujo para las mujeres https://t.co/...,2021-05-17 16:05:28,Estreno de lujo para las mujeres,[https://t.co/oNwKVC5KUW],[],[]
97,"Y aquí está, el nuevo himno de la Champions Le...",2021-05-17 15:44:28,"Y aquí está, el nuevo himno de la Champions Le...",[https://t.co/fSl93rEwHL],[],[]
98,"@luisferpo Que piensan gente de Fútbol, estare...",2021-05-17 15:38:07,"Que piensan gente de Fútbol, estaremos sin Ju...",[],[],[@luisferpo]


In [14]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

In [15]:
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,texto_limpio,links,hashtags,mentions,emojis
0,"Hoy, Fémina Fútbol España presente en la celeb...",2021-05-20 01:12:21,"Hoy, Fémina Fútbol España presente en la celeb...",[https://t.co/ynLsCabBHG],[],[@FCBfemeni],"[💙, ❤️]"
1,⏱️ El LIVERPOOL gana por goleada y asegura su ...,2021-05-20 00:38:28,⏱️ El LIVERPOOL gana por goleada y asegura su ...,[https://t.co/84kZieDbs8],[#PremierLeague],[],[⏱️]
2,¡Pep Guardiola sigue agrandando su historia co...,2021-05-20 00:34:19,¡Pep Guardiola sigue agrandando su historia co...,[https://t.co/BWCvfbR9z7],[],[],"[🧠, ⚽️, 🔝, ⚽️, 🏴󠁧󠁢󠁥󠁮󠁧󠁿, 🏆]"
3,Liverpool ganó y depende de sí para clasificar...,2021-05-20 00:19:04,Liverpool ganó y depende de sí para clasificar...,[https://t.co/y3Lx3wQEhu],[],[],[]
4,La Libertadores men. La puede ver en medio de ...,2021-05-20 00:17:01,La Libertadores men. La puede ver en medio de ...,[],[],[],[]


In [16]:
data['hora'] = data['date'].dt.floor('T').dt.time  
temp = pd.DataFrame(data.hora.value_counts()).reset_index() 
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
30,00:15:00,1
29,00:17:00,1
27,00:19:00,1
62,00:24:00,1
50,00:34:00,1
...,...,...
68,23:42:00,1
60,23:45:00,1
53,23:46:00,1
72,23:51:00,1


In [17]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


In [18]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [20]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

BARRAS

In [29]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values, [])).keys(),Counter(sum(data.emojis.values, [])).values()))) # con keys me muestra llaves(emojis) con values (valor de la lave)
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)

In [30]:
trace = go.Bar(x=temp_emojis.emoji.values,
               y=temp_emojis.cnt.values)

layout = go.Layout(title = 'Emojis más usados')
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

mapa de calor

In [37]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r'[\W\d_]+', " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto

tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado)
tfidf = tfidf_vect.fit_transform(data.texto_limpio.values)

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

voc = [k for k, v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]
temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.values)
temp = pd.DataFrame(temp)
temp

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.013392,0.012481,0.020460,0.007776,0.006488,0.021474,0.047844,0.020460,0.010217,...,0.205891,0.014871,0.025983,0.031039,0.046226,0.040617,0.0,0.013370,0.083454,0.010470
1,0.013392,1.000000,0.014922,0.121573,0.009298,0.113976,0.210808,0.057204,0.121573,0.012216,...,0.014875,0.017780,0.031067,0.037112,0.011122,0.009772,0.0,0.015986,0.073525,0.012518
2,0.012481,0.014922,1.000000,0.117593,0.008665,0.063992,0.056568,0.108249,0.117593,0.011384,...,0.013862,0.016570,0.027021,0.038428,0.031392,0.009107,0.0,0.014897,0.013787,0.010370
3,0.020460,0.121573,0.117593,1.000000,0.014205,0.011851,0.322061,0.171647,1.000000,0.018662,...,0.022725,0.027164,0.047462,0.056698,0.016991,0.014929,0.0,0.024422,0.112327,0.019125
4,0.007776,0.009298,0.008665,0.014205,1.000000,0.004504,0.014908,0.012835,0.014205,0.007093,...,0.008637,0.010324,0.018039,0.021549,0.006458,0.005674,0.0,0.009282,0.008591,0.007269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.040617,0.009772,0.009107,0.014929,0.005674,0.004734,0.063619,0.109218,0.014929,0.041675,...,0.071225,0.010851,0.018959,0.091957,0.234820,1.000000,0.0,0.009756,0.009029,0.007640
96,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
97,0.013370,0.015986,0.014897,0.024422,0.009282,0.007744,0.025632,0.022068,0.024422,0.012195,...,0.014850,0.358580,0.031014,0.037050,0.011103,0.009756,0.0,1.000000,0.185674,0.391008
98,0.083454,0.073525,0.013787,0.112327,0.008591,0.007167,0.086502,0.052854,0.112327,0.011287,...,0.013743,0.166136,0.115483,0.034290,0.010276,0.009029,0.0,0.185674,1.000000,0.011566


In [41]:
trace = go.Heatmap(z=temp.values.tolist(),
                  x=temp.index.values,
                  y=temp.columns.values,
                  colorscale='Oranges')

layout = go.Layout(title='Mapa de Calor entre tweets', #como agregar emojis al titulo?
                  width = 800, height = 800)

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [43]:
data.tweets.iloc[51]

'#PremierLeague \n\nSalió cara la crucial victoria frente al Leicester⬇️ https://t.co/QRkcBbXlO8'

Algoritmo de agrupacion jerarquico

In [44]:
import scipy.cluster.hierarchy as sch
import numpy as np

pairwise_distances = sch.distance.pdist(temp)
linkage = sch.linkage(pairwise_distances, method='ward')
idx_to_cluster_array = sch.fcluster(linkage, pairwise_distances.max() * 0.5, criterion='distance')
idx = np.argsort(idx_to_cluster_array)
temp = temp.copy()
    
temp2 = temp.iloc[idx, :].T.iloc[idx, :]
my_idx = idx_to_cluster_array

temp2.index = ['tweet #'+str(i) for i in temp2.columns]
temp2.columns = temp2.index

In [45]:
trace = go.Heatmap(z=temp2.values.tolist(),
                   x=temp2.index.values,
                   y=temp2.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre tweets (organizado) 🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)

In [46]:
data['cluster'] = my_idx 

for c in data.cluster.value_counts().index:
    df_temp = data[data.cluster==c]
    
    sim = temp2[['tweet #'+str(i) for i in df_temp.index]].mean().mean()
    print(f"***** El cluster {c} tiene {df_temp.shape[0]} tweets con una similaridad de {sim:.2%} *****\n")
    
    tweets_ejemplo = df_temp.head().tweets.values
    for te in tweets_ejemplo:
        print("TWEET")
        print(te)
        print()
    print()
    

***** El cluster 1 tiene 2 tweets con una similaridad de 4.78% *****

TWEET
TECNO Mobile devolvería 5.700 millones de pesos a consumidores colombianos si Manchester City gana la Champions League https://t.co/EzxGEhW4L4

TWEET
TECNO Mobile devolvería 5.700 millones de pesos a consumidores colombianos si Manchester City gana la Champions League https://t.co/ESzewpOGi4


***** El cluster 7 tiene 2 tweets con una similaridad de 4.60% *****

TWEET
🏆🏴󠁧󠁢󠁥󠁮󠁧󠁿 PREMIER LEAGUE 🏴󠁧󠁢󠁥󠁮󠁧󠁿🏆

➡️ El #Liverpool goleó a domicilio al #Burnley y regresa a la zona de #UCL con una fecha por jugarse en la Premier (RESUMEN) ⚽️🥅 https://t.co/oU6izRA9gN ⬅️ https://t.co/P0O1DnCwp5

TWEET
🏆🏴󠁧󠁢󠁥󠁮󠁧󠁿 PREMIER LEAGUE 🏴󠁧󠁢󠁥󠁮󠁧󠁿🏆

➡️ El #Liverpool goleó a domicilio al #Burnley y regresa a la zona de #UCL con una fecha por jugarse en la Premier (RESUMEN) ⚽️🥅 https://t.co/oU6izRiypf ⬅️ https://t.co/0Dz6YIAn4s


***** El cluster 74 tiene 2 tweets con una similaridad de 4.33% *****

TWEET
UEFA Women's Champions League renueva s


TWEET
@danielchavo11 @jcav1704 @DIRECTVSports @Alex_candal El liverpool en 20 años de premier ha ganado 1 liga, el ManU juega la Europa league, esos son sus equipos grandes, el juventus acaba tercero y el milán tiene 8 años sin champions, el citi y el chelsea son nuevos ricos, el madrid y el barça siempre están ahí top x eso son mejores


***** El cluster 86 tiene 1 tweets con una similaridad de 6.55% *****

TWEET
@PintoWanner @Michote91 @Jordisegura191 @InvictosSomos 7 equipos campeones en 6 años (Europa League y Champions League).


***** El cluster 87 tiene 1 tweets con una similaridad de 3.38% *****

TWEET
Hoy perdieron los 4 equipos brasileros en @Libertadores . Quien te conoce Champions League


***** El cluster 88 tiene 1 tweets con una similaridad de 5.14% *****

TWEET
Hace nueve años el Chelsea era campeón de la Champions League. 💙 https://t.co/YFCPYMbTQP


***** El cluster 68 tiene 1 tweets con una similaridad de 3.01% *****

TWEET
@casaleantonio @carlosvaldes5 Saludos, veo 